In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.8 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd

In [7]:
dataset_path="/content/drive/MyDrive/Hacks/LeadersOfDigital NN 2024/data"
# splitted_dataset_path = '/content/drive/MyDrive/Hacks/Atomic2024/splitted_dataset'

In [8]:
users = pd.read_csv(f'{dataset_path}/cntrbtrs_clnts_ops_trn.csv', delimiter=';', encoding='cp1251', dtype={'okato': str, 'pstl_code': str}, parse_dates=['accnt_bgn_date'])
trunc = pd.read_csv(f'{dataset_path}/trnsctns_ops_trn.csv', delimiter=';', encoding='cp1251', parse_dates=['oprtn_date'])

In [25]:
import json
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, CatBoostClassifier, Pool

In [15]:
users.head()
# erly_pnsn_flg

,slctn_nmbr,clnt_id,accnt_id,gndr,brth_yr,prsnt_age,accnt_bgn_date,cprtn_prd_d,erly_pnsn_flg,accnt_status,...,dstrct,city,sttlmnt,pstl_code,okato,phn,email,lk,assgn_npo,assgn_ops
0,0,0x03ED62E56DAEA24B96D6F1252D713286,0xFFB14DE2D28AAD45B08AC54ADDFE6AD2,ж,1960,64,2005-12-21,96,0,Накопительный период,...,NaN,ОМСК Г,NaN,644001,52000000000,нет,нет,нет,нет,нет
1,0,0x15138C4A822F1C45A819FCBE6657B810,0x222F5E5BA3715F418662A3C65B0B051F,м,1954,70,2009-06-29,283,0,Накопительный период,...,NaN,БЕЛОГОРСК Г,NaN,676852,10000000000,нет,нет,нет,нет,нет
2,3,0x943D2C768A4FB38311E659ACF7E960C3,0x943D2C768A4FB38311E659ACF7E960C5,м,1955,69,2006-12-25,88,0,Накопительный период,...,NaN,NaN,NaN,109451,45000000000,нет,нет,нет,нет,нет
3,2,0x9CAE18EC4F30054C83C6C78FF70D124A,0xB7BAA18DCFE2E842A7AF84BBE185265C,ж,1962,62,2013-09-03,1301,0,Накопительный период,...,АЛЬМЕТЬЕВСКИЙ Р-Н,АЛЬМЕТЬЕВСК Г,NaN,423464,92000000000,нет,нет,нет,нет,нет
4,2,0xCB416F5FE5D58E41822B4E9A5209747F,0x68D14914CF3A79408F6A6E097C1A61AF,м,1955,69,2010-12-06,106,0,Накопительный период,...,ВОТКИНСКИЙ Р-Н,NaN,МОЛЧАНЫ Д,427415,94000000000,нет,нет,нет,нет,нет


In [247]:
data = users.copy()

data['gndr'] = data['gndr'].replace({'ж': 0, 'м': 1})

data = data[['accnt_id', 'gndr', 'erly_pnsn_flg']]\
    .astype({
        # 'brth_yr': 'int64',
        # 'prsnt_age': 'int64',
        'gndr': 'int64',
        # 'erly_pnsn_flg': 'int64',
        })

target_column_name = 'erly_pnsn_flg'

X_data = data.drop([target_column_name], axis=1)
y_data = data[target_column_name]

X_train, X_test, y_train, y_test = train_test_split(
    X_data,
    y_data,
    test_size=0.99,
    random_state=42,
    shuffle=True,
    stratify=data[target_column_name])

<ipython-input-247-0fc755be4483>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['gndr'] = data['gndr'].replace({'ж': 0, 'м': 1})


In [252]:
class DataProccesor:
    def process(self, data: pd.DataFrame):
        self.processed_data = data\
            .astype({
                # 'brth_yr': 'int64',
                # 'prsnt_age': 'int64',
                'gndr': 'int64',
                # 'erly_pnsn_flg': 'int64',
                })
        return self.processed_data

In [253]:
data_processor = DataProccesor()
processed_data = data_processor.process(X_test)

In [223]:
class CatBoostTrainer:
    def __init__(self, iterations: int, depth: int):
        self.iterations = iterations
        self.depth = depth

        self.cbc = None

    def train(self, X_train, y_train):
        # categorical_features = train_data.dtypes[
        #     (train_data.dtypes != np.float64) & (train_data.dtypes != np.int64)].index.tolist()

        pooled_train = Pool(data=X_train,
                            label=y_train
                            # cat_features=categorical_features
                            )

        # pooled_test = Pool(data=X_test,
        #                     label=y_test
        #                     # cat_features=categorical_features
        #                     )

        self.cbc = CatBoostClassifier(iterations=self.iterations,
                                 depth=self.depth,
                                 random_seed=42,
                                #  task_type="GPU",
                                #  devices="0:1",
                                 loss_function='CrossEntropy',
                                 eval_metric="F1")

        print("Обучение...")
        self.cbc.fit(pooled_train,
                # eval_set=pooled_test,
                use_best_model=True,
                verbose=True)

    def save(self, weights_path: str):
        self.cbc.save_model(weights_path,
                format="cbm",
                export_parameters=None,
                pool=None)

In [225]:
weights_path = 'weights/catboost.cbm'

model = CatBoostTrainer(
    iterations=10,
    depth=10)

model.train(X_train, y_train)
model.save(weights_path)

Обучение...
0:	learn: 0.0000000	total: 695us	remaining: 6.26ms
1:	learn: 0.0000000	total: 1.67ms	remaining: 6.67ms
2:	learn: 0.0000000	total: 2.39ms	remaining: 5.57ms
3:	learn: 0.0000000	total: 3.15ms	remaining: 4.72ms
4:	learn: 0.0000000	total: 3.88ms	remaining: 3.88ms
5:	learn: 0.0000000	total: 4.57ms	remaining: 3.04ms
6:	learn: 0.0000000	total: 5.44ms	remaining: 2.33ms
7:	learn: 0.0000000	total: 6.39ms	remaining: 1.6ms
8:	learn: 0.0000000	total: 7.11ms	remaining: 789us
9:	learn: 0.0000000	total: 7.83ms	remaining: 0us


You should provide test set for use best model. use_best_model parameter has been switched to false value.


In [260]:
class CatBoostPredictor:
    def __init__(self, ):
        self.cbc = CatBoostClassifier()

        self.accnt_ids = None
        self.predictions = None
        self.submission = None

    def load(self, weights_path: str):
        self.cbc.load_model(weights_path, format='cbm')

    def predict(self, X: pd.DataFrame):
        self.accnt_ids = X[['accnt_id']].copy()

        pooled = Pool(data=X.drop(['accnt_id'], axis=1))

        self.predictions = self.cbc.predict(pooled)

        return self.predictions

    def build_submission(self, submission_path: str):
        self.submission = self.accnt_ids.copy()
        self.submission['erly_pnsn_flg'] = self.predictions

        self.submission = self.submission.reset_index(drop=True)
        self.submission.to_csv(submission_path, header=True)
        return self.submission

In [262]:
model = CatBoostPredictor()
model.load(weights_path)
predictions = model.predict(processed_data)
submission = model.build_submission('submission.csv')

In [219]:
# model.cbc.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [214]:
# f1_score(y_test, model.cbc.predict(X_test))

0.0

In [ ]:
# part_counts = 5
# data = users['brth_yr', 'prsnt_age', 'pstl_code', 'erly_pnsn_flg'].copy()
# test_size = data.shape[0] // part_counts


# logs = {}
# accuracies, f1_scores, recalls, precisions = [], [], [], []

# for part in range(part_counts):
#     test_data = data.iloc[part * test_size: (part + 1) * test_size, :]
#     train_data = data.drop(test_data.index)